In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
# data = data.drop(['Origin_ID', 'Destination_ID', 'Initial_Origin_Balance', 'Initial_Destination_Balance'], axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('DATATHON_EVENT_DATASET.csv')

In [ ]:
data.head()

In [ ]:
# data = data.drop("Expected_Fraud", axis=1)

In [ ]:
data.dtypes

In [ ]:
data['TransactionAmount'] = data['Final_Origin_Balance'] - data['Initial_Origin_Balance']

In [ ]:
# data = data.drop(['Initial_Origin_Balance', 'Initial_Destination_Balance'], axis=1)

In [ ]:
data.columns

In [ ]:
# # Function to hash Origin_ID
# def hash_origin_id(origin_id):
#     return hashlib.sha256(origin_id.encode()).hexdigest()

# # Apply the function to the Origin_ID column
# data['Hashed_Origin_ID'] = data['Origin_ID'].apply(hash_origin_id).rank(method="dense").astype(int)
# data['Hashed_Destination_ID'] = data['Destination_ID'].apply(hash_origin_id).rank(method="dense").astype(int)

encodables = ['Origin_ID', 'Destination_ID']
encoders = {}
for column in encodables:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    encoders[column] = le

In [ ]:
data.head()

In [ ]:
# data = data.drop(['Origin_ID', 'Destination_ID'], axis=1)

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
data['Destination_ID'].value_counts()

In [ ]:
data['Origin_ID'].value_counts()

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
source_amount = (data['Initial_Origin_Balance'] - data['Final_Origin_Balance'])
destination_amount = (data['Final_Destination_Balance'] - data['Initial_Destination_Balance'])

In [ ]:
validity = (source_amount == destination_amount)
validity.value_counts()


In [ ]:
data[validity == False].head()

In [ ]:
data['source_amount'] = source_amount
data['destination_amount'] = destination_amount

In [ ]:
money_columns = ['Amount', 'source_amount', 'destination_amount']

In [ ]:
data[['Amount', 'source_amount', 'destination_amount']].head(50)

In [ ]:
data[(data['source_amount'].abs() == data['destination_amount'].abs()) & (data['source_amount'] != data['Amount'].abs())][money_columns].head()

In [ ]:
data.head()


In [ ]:
columns_to_drop = [
    'Origin_ID', 'Destination_ID', 'Initial_Origin_Balance', 
    'Initial_Destination_Balance', 'Expected_Fraud', 
    'source_amount', 'destination_amount', 'TransactionAmount'
]

# Drop columns if they exist
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])

data.head()

In [ ]:
columns_to_drop = [
    'Origin_ID', 'Destination_ID', 'Initial_Origin_Balance', 
    'Initial_Destination_Balance', 'Expected_Fraud', 
    'source_amount', 'destination_amount', 'TransactionAmount'
]

# Drop columns if they exist
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])

print(data.head())

In [ ]:
data.columns